In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/football-events/ginf.csv
/kaggle/input/football-events/dictionary.txt
/kaggle/input/football-events/events.csv


In [9]:
import sqlite3
import csv
import re

import pandas as pd # to load column names and their data types
import random, string # to generate random table name if necessary
csv_file = "../input/football-events/events.csv"
df_from_csv = pd.read_csv(csv_file, delimiter=',')
df_from_csv.head()
df_from_csv.dtypes.unique()

def get_table_name(csv_file):
    '''
    Create a table name from CSV file name and convert it to be table name
    allowed by slite3 documentation.
    '''
    # when in CSV file name there are letters too
    regex = re.compile('[^a-z]')
    table_name = csv_file.split("/")[-1].split(".")[0]
    table_name = regex.sub('', table_name)

    # when in CSV file name there aren't any letters
    if table_name == '':
        for i in range(10):
            table_name += random.choice(string.ascii_lowercase)
    return table_name

generated_table_name = get_table_name(csv_file)

def create_table(df_dataset, table_name):
    '''
    The function returns SQL statement "CREATE TABLE" with needed table name
    and its column names along with data types which these columns will store.
    '''
    cols_with_sql_types = []
    for col_name, col_type in df_dataset.dtypes.iteritems():
        if col_type == "int64":
            cols_with_sql_types.append('"' + col_name + '"' + " " + 'INTEGER')
        elif col_type == "float64":
            cols_with_sql_types.append('"' + col_name + '"' + " " + 'REAL')
        else:
            cols_with_sql_types.append('"' + col_name + '"' + " " + 'TEXT')

    final = str(cols_with_sql_types).replace("'", "").replace(']', '').replace('[', '')
    return f'CREATE TABLE "{table_name}" ({final})'

def drop_table_if_exists(table_name):
    '''
    The function returns SQL statement "DROP TABLE IF EXISTS" with needed table name.
    '''

    return f'DROP TABLE IF EXISTS {table_name}'

conn = sqlite3.connect(f'{generated_table_name}.sqlite')
cur = conn.cursor()
cur.execute(f"{drop_table_if_exists(generated_table_name)}")
cur.execute(f"{create_table(df_from_csv, generated_table_name)}")

def insert_into_values(df_dataset, table_name):
    '''
    The function returns SQL statement "INSERT INTO" with needed table name and values.
    '''
    numb_of_columns = len(df_dataset.columns)
    values = str(['?' for i in range(numb_of_columns)]).replace("'", "").replace(']', '').replace('[', '')
    return f'INSERT INTO "{table_name}" VALUES ({values})'

def convert_to_str(df_dataset):
    '''
    The function converts problematic dtypes to strings.
    '''
    for i in df_dataset.select_dtypes(include=['datetime', 'timedelta']):
        df_dataset[i] = df_dataset[i].astype(str)

    return df_dataset
def executemany(df_dataset, table_name):
    with sqlite3.connect(f'{table_name}.sqlite'):
        conn = sqlite3.connect(f'{table_name}.sqlite')
        cur = conn.cursor()

        values = convert_to_str(df_dataset).values.tolist()

        cur.executemany(f"{insert_into_values(df_dataset, table_name)}", values)
        conn.commit()

executemany(df_from_csv, generated_table_name)

df_from_sql = pd.read_sql_query(f'select * from {generated_table_name}', con = conn)
df_from_sql


/tmp/ipykernel_34/477008833.py:36: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, col_type in df_dataset.dtypes.iteritems():


,id_odsp,id_event,sort_order,time,text,event_type,event_type2,side,event_team,opponent,...,player_in,player_out,shot_place,shot_outcome,is_goal,location,bodypart,assist_method,situation,fast_break
0,UFot0hit/,UFot0hit1,1,2,Attempt missed. Mladen Petric (Hamburg) left f...,1,12.0,2,Hamburg SV,Borussia Dortmund,...,None,None,6.0,2.0,0,9.0,2.0,1,1.0,0
1,UFot0hit/,UFot0hit2,2,4,"Corner, Borussia Dortmund. Conceded by Dennis...",2,NaN,1,Borussia Dortmund,Hamburg SV,...,None,None,NaN,NaN,0,NaN,NaN,0,NaN,0
2,UFot0hit/,UFot0hit3,3,4,"Corner, Borussia Dortmund. Conceded by Heiko ...",2,NaN,1,Borussia Dortmund,Hamburg SV,...,None,None,NaN,NaN,0,NaN,NaN,0,NaN,0
3,UFot0hit/,UFot0hit4,4,7,Foul by Sven Bender (Borussia Dortmund).,3,NaN,1,Borussia Dortmund,Hamburg SV,...,None,None,NaN,NaN,0,NaN,NaN,0,NaN,0
4,UFot0hit/,UFot0hit5,5,7,Gokhan Tore (Hamburg) wins a free kick in the ...,8,NaN,2,Hamburg SV,Borussia Dortmund,...,None,None,NaN,NaN,0,2.0,NaN,0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
941004,z5L2OT5E/,z5L2OT5E123,123,92,Lucas Torreira (Sampdoria) wins a free kick in...,8,NaN,2,Sampdoria,Atalanta,...,None,None,NaN,NaN,0,2.0,NaN,0,NaN,0
941005,z5L2OT5E/,z5L2OT5E124,124,93,"Corner, Sampdoria. Conceded by Andrea Masiello.",2,NaN,2,Sampdoria,Atalanta,...,None,None,NaN,NaN,0,NaN,NaN,0,NaN,0
941006,z5L2OT5E/,z5L2OT5E125,125,93,Attempt missed. Fabio Quagliarella (Sampdoria)...,1,12.0,2,Sampdoria,Atalanta,...,None,None,8.0,2.0,0,9.0,1.0,1,3.0,0
941007,z5L2OT5E/,z5L2OT5E126,126,94,Alberto Grassi (Atalanta) wins a free kick on ...,8,NaN,1,Atalanta,Sampdoria,...,None,None,NaN,NaN,0,4.0,NaN,0,NaN,0


In [10]:
df_from_sql = pd.read_sql_query(f'SELECT COUNT(*) AS Total_Goals FROM {generated_table_name}', con = conn)
df_from_sql

,Total_Goals
0,24446


In [11]:
#through ball
df_from_sql = pd.read_sql_query(f'SELECT COUNT(*) AS Through_Ball FROM {generated_table_name} WHERE assist_method = 4' , con = conn)
df_from_sql

,Through_Ball
0,1920


In [12]:
df_from_sql = pd.read_sql_query(f'SELECT COUNT(*) AS Cross FROM {generated_table_name} WHERE assist_method = 2' , con = conn)
df_from_sql

,Cross
0,5143


In [13]:
df_from_sql = pd.read_sql_query(f'SELECT COUNT(*) AS Set_Piece FROM {generated_table_name} WHERE situation = 2' , con = conn)
df_from_sql

,Set_Piece
0,3671


In [14]:
df_from_sql = pd.read_sql_query(f'SELECT COUNT(*) AS Open_Play FROM {generated_table_name} WHERE situation = 1' , con = conn)
df_from_sql

,Open_Play
0,17308


In [15]:
df_from_sql = pd.read_sql_query(f'SELECT COUNT(*) AS Left_Foot FROM {generated_table_name} WHERE bodypart = 2 ' , con = conn)
df_from_sql

,Left_Foot
0,6758


In [16]:
df_from_sql = pd.read_sql_query(f'SELECT COUNT(*) AS Right_Foot FROM {generated_table_name} WHERE bodypart = 1' , con = conn)
df_from_sql

,Right_Foot
0,13451


In [17]:
df_from_sql = pd.read_sql_query(f'SELECT COUNT(*) AS Head FROM {generated_table_name} WHERE bodypart = 3' , con = conn)
df_from_sql

,Head
0,4236


In [18]:
df_from_sql = pd.read_sql_query(f'SELECT COUNT(*) AS On_Target FROM {generated_table_name} WHERE shot_outcome = 1', con = conn)
df_from_sql

,On_Target
0,23816


In [19]:
df_from_sql = pd.read_sql_query(f'SELECT COUNT(*) AS Off_Target FROM {generated_table_name} WHERE shot_outcome = 2' , con = conn)
df_from_sql

,Off_Target
0,0


In [23]:
df_from_sql = pd.read_sql_query(f'SELECT COUNT(*) AS Off_Target FROM {generated_table_name} WHERE (bodypart = 1) AND (situation = 1)'  , con = conn)
df_from_sql

,Off_Target
0,10010
